In [ ]:
import logging
import asyncio
import re
import nest_asyncio
from telegram import Update, ReplyKeyboardRemove
from telegram.ext import (
    Application, CommandHandler, MessageHandler, 
    filters, ContextTypes, ConversationHandler
)
from playwright.async_api import async_playwright
import google.generativeai as genai

# Required for Jupyter environments
nest_asyncio.apply()

# --- Configuration ---
TELEGRAM_TOKEN = ""
genai.configure(api_key="")
model = genai.GenerativeModel('gemini-2.5-flash')

# Conversation States
ENTERING_ID, ENTERING_PW, ENTERING_OTP = range(3)

# Global dictionary to store playwright objects per user
user_sessions = {}

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

### --- Helper Functions ---

async def cleanup(chat_id):
    """Closes browsers and stops Playwright for a specific user."""
    if chat_id in user_sessions:
        print(f"Cleaning up session for {chat_id}")
        session = user_sessions[chat_id]
        try:
            await session["browser"].close()
            await session["pw"].stop()
        except Exception as e:
            print(f"Cleanup error: {e}")
        del user_sessions[chat_id]

async def solve_bbdc_captcha(page):
    """Solves the BBDC captcha using Gemini AI."""
    for attempt in range(5):
        captcha_selector = 'div.form-captcha-image-wrapper div.v-image__image--cover'
        await page.wait_for_selector(captcha_selector, state="visible")
        captcha_element = page.locator(captcha_selector).last
        style_attribute = await captcha_element.get_attribute("style")
        match = re.search(r'base64,([^&"\s\)]+)', style_attribute)
        if not match:
            await page.get_by_text("Refresh").click()
            continue

        base64_str = match.group(1)
        prompt = "Identify the 5-6 alphanumeric characters in this noisy image. Output ONLY the characters."
        response = await model.generate_content_async([prompt, {'mime_type': 'image/png', 'data': base64_str}])
        captcha_text = response.text.strip().replace(" ", "")
        
        await page.get_by_label("Captcha").fill(captcha_text)
        await page.get_by_role("button", name="Verify").click()
        await asyncio.sleep(2)
        if not await page.get_by_text("Captcha is required").is_visible():
            return True
        await page.get_by_text("Refresh").click()
    return False

### --- Telegram Handler Functions ---

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("🚗 BBDC Bot Ready. Use /check to start.")

async def check_start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("👤 Please enter your BBDC Login ID:")
    return ENTERING_ID

async def get_login_id(update: Update, context: ContextTypes.DEFAULT_TYPE):
    context.user_data['login_id'] = update.message.text
    await update.message.reply_text("🔑 Got it. Now enter your Password:")
    return ENTERING_PW

async def get_password(update: Update, context: ContextTypes.DEFAULT_TYPE):
    context.user_data['password'] = update.message.text
    chat_id = update.effective_chat.id
    await update.message.reply_text("🌐 Opening browser. Please wait...")

    pw = await async_playwright().start()
    browser = await pw.chromium.launch(headless=True)
    page = await browser.new_page()
    user_sessions[chat_id] = {"pw": pw, "browser": browser, "page": page}

    try:
        await page.goto("https://booking.bbdc.sg/#/login")
        await page.get_by_label("Login ID").fill(context.user_data['login_id'])
        await page.get_by_label("Password").and_(page.get_by_role("textbox")).fill(context.user_data['password'])
        await page.get_by_role("button", name="Access to Booking System").click()
        await asyncio.sleep(4)

        if await page.get_by_text("Send OTP").is_visible():
            await page.get_by_text("Send OTP", exact=True).click()
            await update.message.reply_text("📲 OTP Sent! Please reply with the 6-digit code.")
            return ENTERING_OTP
        elif await page.locator('div.form-captcha-image-wrapper').is_visible():
            await update.message.reply_text("🧩 Solving Captcha...")
            if await solve_bbdc_captcha(page):
                return await scrape_results(update, context)
            else:
                await update.message.reply_text("❌ Captcha failed.")
                await cleanup(chat_id)
                return ConversationHandler.END
        else:
            # If no security check, try scraping directly
            return await scrape_results(update, context)
    except Exception as e:
        await update.message.reply_text(f"⚠️ Login Error: {str(e)}")
        await cleanup(chat_id)
        return ConversationHandler.END

async def handle_otp(update: Update, context: ContextTypes.DEFAULT_TYPE):
    chat_id = update.effective_chat.id
    otp = update.message.text
    session = user_sessions.get(chat_id)
    
    if not session:
        await update.message.reply_text("Session lost. Please /check again.")
        return ConversationHandler.END

    page = session["page"]
    try:
        await page.get_by_label("OTP").fill(otp)
        await page.get_by_role("button", name=re.compile("Login|Verify|Confirm", re.I)).click()
        await asyncio.sleep(3)
        return await scrape_results(update, context)
    except Exception as e:
        await update.message.reply_text(f"⚠️ OTP Error: {str(e)}")
        await cleanup(chat_id)
        return ConversationHandler.END

async def scrape_results(update: Update, context: ContextTypes.DEFAULT_TYPE):
    chat_id = update.effective_chat.id
    page = user_sessions[chat_id]["page"]

    try:
        await page.get_by_text("Booking", exact=True).click()
        await page.get_by_text("Practical", exact=True).click()
        await asyncio.sleep(2)

        # Bug reset logic
        if await page.get_by_text("not required to attend practical").is_visible():
            await page.get_by_text("Booking", exact=True).click()
            await page.get_by_text("Practical", exact=True).click()

        await page.wait_for_selector('div.sessionList', timeout=15000)
        
        results = []
        session_lists = page.locator('div.sessionList')
        count = await session_lists.count()

        for i in range(count):
            container = session_lists.nth(i)
            date_text = await container.locator('p.title span.left').inner_text()
            cards = container.locator('div.sessionContent-web div.sessionCard')
            
            slots = []
            for j in range(await cards.count()):
                details = await cards.nth(j).locator('p').all_text_contents()
                slots.append(f"• {' | '.join([d.strip() for d in details if d.strip()])}")
            
            results.append(f"📅 *{date_text}*\n" + "\n".join(slots))

        final_msg = "\n\n".join(results) if results else "No slots found. ☹️"
        await update.message.reply_text(final_msg, parse_mode='Markdown')

    except Exception as e:
        await update.message.reply_text(f"⚠️ Scrape Error: {str(e)}")
    
    finally:
        await cleanup(chat_id)
    return ConversationHandler.END

async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await cleanup(update.effective_chat.id)
    await update.message.reply_text("Session cancelled.", reply_markup=ReplyKeyboardRemove())
    return ConversationHandler.END

### --- Execution ---

async def main():
    application = Application.builder().token(TELEGRAM_TOKEN).build()

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('check', check_start)],
        states={
            ENTERING_ID: [MessageHandler(filters.TEXT & ~filters.COMMAND, get_login_id)],
            ENTERING_PW: [MessageHandler(filters.TEXT & ~filters.COMMAND, get_password)],
            ENTERING_OTP: [MessageHandler(filters.TEXT & ~filters.COMMAND, handle_otp)],
        },
        fallbacks=[CommandHandler('cancel', cancel)],
    )

    application.add_handler(CommandHandler("start", start))
    application.add_handler(conv_handler)
    
    print("Bot is starting...")
    async with application:
        await application.initialize()
        await application.start()
        await application.updater.start_polling()
        while True:
            await asyncio.sleep(1)

# Entry point for Jupyter
try:
    loop = asyncio.get_event_loop()
    if loop.is_running():
        loop.create_task(main())
    else:
        asyncio.run(main())
except RuntimeError:
    asyncio.run(main())